# Collect all Paths 
This does not need to be ran anymore we have the list in paths.txt now. Just read that into simulation_paths and you are good to go with collecting data

In [1]:
import s3fs
import zarr
import json
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import pickle
from pprint import pprint
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
endpoint = 'https://wifire-data.sdsc.edu:9000'
access_key = os.getenv("ACCESS_KEY")
secret_key = os.getenv("SECRET_KEY")

fs = s3fs.S3FileSystem(key=access_key,
    secret=secret_key,
    client_kwargs={
        'endpoint_url': endpoint,
        'verify': False
    },
    skip_instance_cache=False
)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

name = 'quicfire.zarr'
bucket = 'burnpro3d/d'

root = list(fs.ls(bucket))

simulation_paths = []
print("successfully authenticated")

successfully authenticated


In [7]:
def get_paths(rangestart,rangeend):
    global fs, bucket
    paths = fs.ls(bucket)
    for path in paths[rangestart:rangeend]:
        paths = fs.ls(path)
        for path in paths:
            get_child_directories(path)

def get_child_directories(path):
    global fs
    global simulation_paths
    paths = fs.ls(path)
    for p in paths:
        if "run_" in p:
            simulation_paths.append(p)

# def read_paths():
#     csv_file = "csv_files/successful_bp3d_runs.csv"
#     df = pd.read_csv(csv_file, index_col=0)
#     paths_df = df["s3_path"]
#     paths_list = paths_df.tolist()
#     return paths_list

def read_paths():
    paths = []
    with open("paths.txt","r") as f:
        paths = f.read().splitlines() 
    return paths


In [8]:
simulation_paths = read_paths()
print("simulation paths length:", len(simulation_paths))

FileNotFoundError: [Errno 2] No such file or directory: 'paths.txt'

# Now Extract Data

In [5]:
KEEP_ATTRIBUTES = {
    'path': lambda d: None,
    'time_scraped': lambda d: None,
    'canopy_moisture': lambda d: d['canopy_moisture'],
    'dz':lambda d: d['dz'],
    'extent': lambda d: d['extent'],
    'extent_fmt': lambda d: d['extent_fmt'],
    'fire_grid': lambda d: d['fire_grid'],
    'fuel': lambda d: d['fuel'],
    'ignition': lambda d: d['ignition'],
    'output': lambda d: d['output'],
    'resolution': lambda d: d['resolution'],
    'resolution_units': lambda d: d['resolution_units'],
    'run_binary': lambda d: d['run_binary'],
    'run_end': lambda d: d['run_end'],
    'run_max_mem_rss_bytes': lambda d: d['run_max_mem_rss_bytes'],
    'run_start': lambda d: d['run_start'],
    'seed': lambda d: d['seed'],
    'sim_time': lambda d: d['sim_time'],
    'surface_moisture': lambda d: d['surface_moisture'],
    'threads': lambda d: d['threads'],
    'timestep': lambda d: d['timestep'],
    'topo': lambda d: d['topo'],
    'wind_direction': lambda d: d['wind_direction'],
    'wind_speed': lambda d: d['wind_speed']
}
filenotfound = []

def get_df_chunk(stop):
    with open("vars.txt", "r") as file:
        start = int(file.read())
        print("start: line" , start)
    global simulation_paths, KEEP_ATTRIBUTES, incomplete, filenotfound

    df = pd.DataFrame([], columns=KEEP_ATTRIBUTES.keys())
    time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
    i = 0
    if stop > len(paths):
        stop = len(paths)
    for p in tqdm(simulation_paths[start:stop]):
        try:
            with fs.open(p + '/' + name + '/.zattrs') as f:
                data=json.load(f)
        except:
            filenotfound.append(p)
            print("FileNotFound error on path {",p,"}")
            continue
            
        row = []
        for k,expr in KEEP_ATTRIBUTES.items():
            try:
                value = KEEP_ATTRIBUTES[k](data)
            except KeyError:
                value = None
            row.append(value)
        row[0] = p
        row[1] = time
        df.loc[i] = row
        i+=1
    with open("vars.txt", "w") as f:
            f.write(str(stop))
            print("\nRead from ", start, " to ", stop,"\n")
    return df

pd.set_option('display.max_columns', None)
print(filenotfound)

[]


In [6]:
simulation_paths = read_paths()
df = get_df_chunk(1889)

start: line 0


100%|█████████████████████████████████████████████████████████| 1889/1889 [05:01<00:00,  6.26it/s]


Read from  0  to  1889 



In [7]:
df

,path,time_scraped,canopy_moisture,dz,extent,extent_fmt,fire_grid,fuel,ignition,output,resolution,resolution_units,run_binary,run_end,run_max_mem_rss_bytes,run_start,seed,sim_time,surface_moisture,threads,timestep,topo,wind_direction,wind_speed
0,burnpro3d/d/88/39/run_883966ce-ba84-4dfd-ac6c-...,2023-12-26T00:30:38.559821,0.90,None,"[-1959092.0, 1310426.0, -1958552.0, 1309880.0]","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-14T10:21:23.901140-07:00,401160,2023-06-14T10:15:55.918969-07:00,-1,4005,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",120.0,2.235
1,burnpro3d/d/f9/e5/run_f9e5fe89-7b72-4bd5-8e76-...,2023-12-26T00:30:38.559821,0.90,None,"[-1959092.0, 1310426.0, -1958552.0, 1309880.0]","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-14T10:29:39.092975-07:00,401184,2023-06-14T10:20:47.772654-07:00,-1,4005,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",90.0,2.235
2,burnpro3d/d/ff/50/run_ff50256d-8fd9-438b-a322-...,2023-12-26T00:30:38.559821,0.90,None,"[-1959092.0, 1310426.0, -1958552.0, 1309880.0]","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-14T10:29:42.945253-07:00,401132,2023-06-14T10:20:56.196752-07:00,-1,4005,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",60.0,2.235
3,burnpro3d/d/70/e2/run_70e2d0fd-9ffe-4a29-92be-...,2023-12-26T00:30:38.559821,0.90,None,"[-2022346.0, 1581420.0, -2020914.0, 1580036.0]","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-23T16:34:06.894350-07:00,2428216,2023-06-23T14:18:18.583415-07:00,-1,13493,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",90.0,6.705
4,burnpro3d/d/1a/31/run_1a31d7ee-14c6-4729-ab3e-...,2023-12-26T00:30:38.559821,0.90,None,"[-2022346.0, 1581420.0, -2020914.0, 1580036.0]","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-23T17:19:06.935054-07:00,2428036,2023-06-23T14:18:15.973481-07:00,-1,13493,0.15,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",90.0,4.470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,burnpro3d/d/2a/86/run_2a86e01e-d7d0-45e2-96ee-...,2023-12-26T00:30:38.559821,0.90,None,"[1450216.0, 763242.0, 1451310.0, 761958.0]","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-10-20T07:38:32.646165-07:00,2269772,2023-10-20T05:51:29.423214-07:00,-1,7251,0.08,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",331.0,4.470
1885,burnpro3d/d/7c/5e/run_7c5e25e2-050f-408d-bddc-...,2023-12-26T00:30:38.559821,0.90,None,"[1450216.0, 763242.0, 1451310.0, 761958.0]","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-10-20T07:17:50.576771-07:00,2269556,2023-10-20T05:51:28.792834-07:00,-1,7251,0.08,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",1.0,4.470
1886,burnpro3d/d/be/0a/run_be0af646-0d10-4947-a72b-...,2023-12-26T00:30:38.559821,0.86,None,"[-2097480.0, 2037504.0, -2096674.0, 2036700.0]","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-10-13T18:30:36.641747-07:00,3913120,2023-10-13T17:38:01.089893-07:00,-1,4139,0.12,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",126.0,1.788
1887,burnpro3d/d/9a/4c/run_9a4ca7a0-de7a-479f-aae8-...,2023-12-26T00:30:38.559821,0.86,None,"[-2097480.0, 2037504.0, 

## remove NaN rows

In [8]:
# crucial columns that need to have data. If they do not, that means the run failed somewhere
time_cols = ['run_start', 'run_end', 'sim_time']

# Create a new DataFrame that includes rows with NA values in 'start', 'stop', or 'runtime'
# Then store it in a csv file called na_times
na_mask = df[time_cols].isna()
na_rows_df = df[na_mask.any(axis=1)]
if len(na_rows_df > 0):
    na_rows_df.to_csv('csv_files/na_times.csv', mode='a')

# Drop columns with NA values in any of the time columns
df = df.dropna(subset=time_cols)

# Rename the 'run_end' column to stop and run_start column to 'start'
df = df.rename(columns={"run_end": "stop", "run_start": "start"})
df = df.drop(columns=["ignition","fuel"], axis=1)

df

,path,time_scraped,canopy_moisture,dz,extent,extent_fmt,fire_grid,output,resolution,resolution_units,run_binary,stop,run_max_mem_rss_bytes,start,seed,sim_time,surface_moisture,threads,timestep,topo,wind_direction,wind_speed
0,burnpro3d/d/88/39/run_883966ce-ba84-4dfd-ac6c-...,2023-12-26T00:30:38.559821,0.90,None,"[-1959092.0, 1310426.0, -1958552.0, 1309880.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-14T10:21:23.901140-07:00,401160,2023-06-14T10:15:55.918969-07:00,-1,4005,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",120.0,2.235
1,burnpro3d/d/f9/e5/run_f9e5fe89-7b72-4bd5-8e76-...,2023-12-26T00:30:38.559821,0.90,None,"[-1959092.0, 1310426.0, -1958552.0, 1309880.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-14T10:29:39.092975-07:00,401184,2023-06-14T10:20:47.772654-07:00,-1,4005,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",90.0,2.235
2,burnpro3d/d/ff/50/run_ff50256d-8fd9-438b-a322-...,2023-12-26T00:30:38.559821,0.90,None,"[-1959092.0, 1310426.0, -1958552.0, 1309880.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-14T10:29:42.945253-07:00,401132,2023-06-14T10:20:56.196752-07:00,-1,4005,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",60.0,2.235
3,burnpro3d/d/70/e2/run_70e2d0fd-9ffe-4a29-92be-...,2023-12-26T00:30:38.559821,0.90,None,"[-2022346.0, 1581420.0, -2020914.0, 1580036.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-23T16:34:06.894350-07:00,2428216,2023-06-23T14:18:18.583415-07:00,-1,13493,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",90.0,6.705
4,burnpro3d/d/1a/31/run_1a31d7ee-14c6-4729-ab3e-...,2023-12-26T00:30:38.559821,0.90,None,"[-2022346.0, 1581420.0, -2020914.0, 1580036.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-23T17:19:06.935054-07:00,2428036,2023-06-23T14:18:15.973481-07:00,-1,13493,0.15,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",90.0,4.470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,burnpro3d/d/2a/86/run_2a86e01e-d7d0-45e2-96ee-...,2023-12-26T00:30:38.559821,0.90,None,"[1450216.0, 763242.0, 1451310.0, 761958.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-10-20T07:38:32.646165-07:00,2269772,2023-10-20T05:51:29.423214-07:00,-1,7251,0.08,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",331.0,4.470
1885,burnpro3d/d/7c/5e/run_7c5e25e2-050f-408d-bddc-...,2023-12-26T00:30:38.559821,0.90,None,"[1450216.0, 763242.0, 1451310.0, 761958.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-10-20T07:17:50.576771-07:00,2269556,2023-10-20T05:51:28.792834-07:00,-1,7251,0.08,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",1.0,4.470
1886,burnpro3d/d/be/0a/run_be0af646-0d10-4947-a72b-...,2023-12-26T00:30:38.559821,0.86,None,"[-2097480.0, 2037504.0, -2096674.0, 2036700.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-10-13T18:30:36.641747-07:00,3913120,2023-10-13T17:38:01.089893-07:00,-1,4139,0.12,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",126.0,1.788
1887,burnpro3d/d/9a/4c/run_9a4ca7a0-de7a-479f-aae8-...,2023-12-26T00:30:38.559821,0.86,None,"[-2097480.0, 2037504.0, -2096674.0, 2036700.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0

In [13]:
read_df = pd.read_csv("csv_files/unfiltered.csv", index_col=0)
read_df

,Unnamed: 0,path,time_scraped,canopy_moisture,dz,extent,extent_fmt,fire_grid,output,resolution,resolution_units,run_binary,stop,run_max_mem_rss_bytes,start,seed,sim_time,surface_moisture,threads,timestep,topo,wind_direction,wind_speed
0,0,burnpro3d/d/88/39/run_883966ce-ba84-4dfd-ac6c-...,2023-12-26T00:30:38.559821,0.90,NaN,"[-1959092.0, 1310426.0, -1958552.0, 1309880.0]","[[x1, y1], [x2, y2]]",NaN,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-14T10:21:23.901140-07:00,401160,2023-06-14T10:15:55.918969-07:00,-1,4005,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",120.0,2.235
1,1,burnpro3d/d/f9/e5/run_f9e5fe89-7b72-4bd5-8e76-...,2023-12-26T00:30:38.559821,0.90,NaN,"[-1959092.0, 1310426.0, -1958552.0, 1309880.0]","[[x1, y1], [x2, y2]]",NaN,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-14T10:29:39.092975-07:00,401184,2023-06-14T10:20:47.772654-07:00,-1,4005,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",90.0,2.235
2,2,burnpro3d/d/ff/50/run_ff50256d-8fd9-438b-a322-...,2023-12-26T00:30:38.559821,0.90,NaN,"[-1959092.0, 1310426.0, -1958552.0, 1309880.0]","[[x1, y1], [x2, y2]]",NaN,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-14T10:29:42.945253-07:00,401132,2023-06-14T10:20:56.196752-07:00,-1,4005,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",60.0,2.235
3,3,burnpro3d/d/70/e2/run_70e2d0fd-9ffe-4a29-92be-...,2023-12-26T00:30:38.559821,0.90,NaN,"[-2022346.0, 1581420.0, -2020914.0, 1580036.0]","[[x1, y1], [x2, y2]]",NaN,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-23T16:34:06.894350-07:00,2428216,2023-06-23T14:18:18.583415-07:00,-1,13493,0.10,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",90.0,6.705
4,4,burnpro3d/d/1a/31/run_1a31d7ee-14c6-4729-ab3e-...,2023-12-26T00:30:38.559821,0.90,NaN,"[-2022346.0, 1581420.0, -2020914.0, 1580036.0]","[[x1, y1], [x2, y2]]",NaN,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-06-23T17:19:06.935054-07:00,2428036,2023-06-23T14:18:15.973481-07:00,-1,13493,0.15,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",90.0,4.470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,1884,burnpro3d/d/2a/86/run_2a86e01e-d7d0-45e2-96ee-...,2023-12-26T00:30:38.559821,0.90,NaN,"[1450216.0, 763242.0, 1451310.0, 761958.0]","[[x1, y1], [x2, y2]]",NaN,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-10-20T07:38:32.646165-07:00,2269772,2023-10-20T05:51:29.423214-07:00,-1,7251,0.08,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",331.0,4.470
1885,1885,burnpro3d/d/7c/5e/run_7c5e25e2-050f-408d-bddc-...,2023-12-26T00:30:38.559821,0.90,NaN,"[1450216.0, 763242.0, 1451310.0, 761958.0]","[[x1, y1], [x2, y2]]",NaN,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-10-20T07:17:50.576771-07:00,2269556,2023-10-20T05:51:28.792834-07:00,-1,7251,0.08,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",1.0,4.470
1886,1886,burnpro3d/d/be/0a/run_be0af646-0d10-4947-a72b-...,2023-12-26T00:30:38.559821,0.86,NaN,"[-2097480.0, 2037504.0, -2096674.0, 2036700.0]","[[x1, y1], [x2, y2]]",NaN,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.2-ucsd-rel,2023-10-13T18:30:36.641747-07:00,3913120,2023-10-13T17:38:01.089893-07:00,-1,4139,0.12,1,600,"{'topo_calcs': False, 'total_startup_iters': 500}",126.0,1.788
1887,1887,burnpro3d/d/9a/4c/run_9a4ca7a0-de7a-479f-aae8-...,2023-12-26T00:30:38.559821,0.86,NaN,"[-2097480.0, 2037504.0, -2096674.0, 2036700.0]","[[x1, y1], [x2, y2]]",NaN,"{'emissions': False, 'energy_

# Write df data to a file

In [12]:
if len(df > 0):
    df.to_csv('csv_files/unfiltered.csv', mode='a')

# Push Data to Influx

In [12]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client import InfluxDBClient, Point, WriteOptions

bucket = "devin"
url= "http://localhost:8086"
token = os.getenv("INFLUXDB_TOKEN")
org = "SDSC internship"
bucket = "perf-data-testing"

client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org
)

# Write script
write_api = client.write_api(write_options=SYNCHRONOUS)

# p = influxdb_client.Point("hi").tag("location", "Czech").field("height",6).field("age",8)
# write_api.write(bucket=bucket, org=org, record=p)

ModuleNotFoundError: No module named 'influxdb_client'

In [13]:
'2023-02-24T11:37:45.285185-08:00'
datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")

'2023-10-25T21:14:37.922244'

In [14]:

# df

# df = df.drop(columns=['ignition','fuel'])
# df
# DataFrame must have the timestamp column as an index for the client. 


write_api.write(bucket, record=df, data_frame_measurement_name='bp3d-run',
                    data_frame_tag_columns=[])

NameError: name 'write_api' is not defined

In [316]:
df

,path,time_scraped,canopy_moisture,dz,extent,extent_fmt,fire_grid,output,resolution,resolution_units,run_binary,stop,run_max_mem_rss_bytes,seed,sim_time,surface_moisture,threads,timestep,topo,wind_direction,wind_speed
start,,,,,,,,,,,,,,,,,,,,,
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/08/run_0008c20d-b78b-4a33-bd01-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,2023-07-27 14:22:05.476561+00:00,6772472,-1,3200,0.35,8,25,None,225.000000,3.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0b/run_000baafc-fe1a-48a1-90c5-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[-2048049, 1866939, -2046749, 1866139]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[1, 2, 2]",meters,/quicfire/quicfire_Jan2022,2023-07-27 14:22:05.476561+00:00,3561948,-1,2100,0.35,None,25,None,180.000000,8.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0d/run_000d836a-e4d0-46c2-8ce6-...,2023-07-27 14:22:05.476561+00:00,1.5,None,"[939794.0, 1235562.0, 941860.0, 1233570.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[1.0, 2.0, 2.0]",meters,/quicfire/quicfire-v4.0.1,2023-07-27 14:22:05.476561+00:00,None,-1,18600,0.05,1,None,None,245.000000,1.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0d/run_000de038-3005-461e-a6d6-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[0, 600, 600, 0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2]",meters,/quicfire/quicfire-v5.1.1,2023-07-27 14:22:05.476561+00:00,428896,-1,600,0.35,1,100,{'total_startup_iters': 0},270.000000,6.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/00/0e/run_000e95ef-a111-4371-a698-...,2023-07-27 14:22:05.476561+00:00,120.0,None,"[1335078.0, 1098858.0, 1336356.0, 1096868.0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2.0, 2.0, 1.0]",meters,/quicfire/quicfire-v5.1.1,2023-07-27 14:22:05.476561+00:00,None,-1,15963,7.00,1,None,None,335.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/01/02/run_0102f75c-8d04-46b6-86ea-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[1120178.154045004, 913143.8979546748, 1120778...","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[1, 2, 2]",meters,/quicfire/quicfire_Jan2022,2023-07-27 14:22:05.476561+00:00,523068,-1,1000,0.00,1,10,None,264.614715,4.550601
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/01/06/run_01062a0e-e7ee-4420-aea2-...,2023-07-27 14:22:05.476561+00:00,0.1,None,"[0, 500, 500, 0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2]",meters,/quicfire/quicfire-v5.0.0,2023-07-27 14:22:05.476561+00:00,301760,-1,5,0.05,1,100,None,270.000000,5.000000
2023-07-27 14:22:05.476561+00:00,burnpro3d/d/01/08/run_010827f3-e59d-422e-9f42-...,2023-07-27 14:22:05.476561+00:00,1.0,None,"[0, 600, 600, 0]","[[x1, y1], [x2, y2]]",None,"{'emissions': False, 'energy_atmos': True, 'fi...","[2, 2]",meters,/ceph/QF_Versions/QF_1-6-22/source_code/compil...,2023-07-27 14:22:05.476561+00:00,654512,-1,600,0.10,None,1,{'total_startup_iters': 0},280.000000,10.000000


# Now Query for the Data we entered

In [321]:
query_api = client.query_api()

query= '''
from(bucket: "perf-data-testing")
|> range(start:-5d, stop: now())
|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
|> group(columns: ["fuel"])
'''

data = client.query_api().query_data_frame(org=org, query=query)
data

,result,table,_start,_stop,_time,_measurement,canopy_moisture,dz,extent,extent_fmt,fire_grid,fuel,ignition,output,path,resolution,resolution_units,run_binary,run_max_mem_rss_bytes,seed,sim_time,stop,surface_moisture,threads,time_scraped,timestep,topo,wind_direction,wind_speed
0,_result,0,2023-07-22 21:23:53.559796+00:00,2023-07-27 21:23:53.559796+00:00,2023-07-27 14:22:05.476561+00:00,bp3d-run,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...",burnpro3d/d/01/15/run_0115cd68-a700-4fcd-ac79-...,"[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,None,-1,3200,2023-07-27 14:22:05.476561+00:00,0.35,8,2023-07-27 14:22:05.476561+00:00,None,None,195.0,2.0
1,_result,1,2023-07-22 21:23:53.559796+00:00,2023-07-27 21:23:53.559796+00:00,2023-07-27 14:16:00.039445+00:00,bp3d-run,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,"{'density': None, 'dx': 2, 'dy': 2, 'height': ...","{'dat': [' igntype= 4\n', ' &aeria...","{'emissions': False, 'energy_atmos': False, 'f...",burnpro3d/d/01/15/run_0115cd68-a700-4fcd-ac79-...,"[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,None,-1,3200,2023-07-27 14:16:00.039445+00:00,0.35,8,2023-07-27 14:16:00.039445+00:00,None,None,195.0,2.0


In [324]:
data

,result,table,_start,_stop,_time,_measurement,canopy_moisture,dz,extent,extent_fmt,fire_grid,fuel,ignition,output,path,resolution,resolution_units,run_binary,run_max_mem_rss_bytes,seed,sim_time,stop,surface_moisture,threads,time_scraped,timestep,topo,wind_direction,wind_speed
0,_result,0,2023-07-22 21:23:53.559796+00:00,2023-07-27 21:23:53.559796+00:00,2023-07-27 14:22:05.476561+00:00,bp3d-run,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,None,None,"{'emissions': False, 'energy_atmos': False, 'f...",burnpro3d/d/01/15/run_0115cd68-a700-4fcd-ac79-...,"[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,None,-1,3200,2023-07-27 14:22:05.476561+00:00,0.35,8,2023-07-27 14:22:05.476561+00:00,None,None,195.0,2.0
1,_result,1,2023-07-22 21:23:53.559796+00:00,2023-07-27 21:23:53.559796+00:00,2023-07-27 14:16:00.039445+00:00,bp3d-run,1.0,None,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,"{'density': None, 'dx': 2, 'dy': 2, 'height': ...","{'dat': [' igntype= 4\n', ' &aeria...","{'emissions': False, 'energy_atmos': False, 'f...",burnpro3d/d/01/15/run_0115cd68-a700-4fcd-ac79-...,"[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,None,-1,3200,2023-07-27 14:16:00.039445+00:00,0.35,8,2023-07-27 14:16:00.039445+00:00,None,None,195.0,2.0


In [322]:
LATEST_VERSION = 'quicfire-v5.2-ucsd-rel'
# drop all multi threaded and none rows
# keep a record of how many left


In [ ]:
[-2048049, 1866939, -2046749, 1866139]
[x1 y1 x2 y2]
A = (x1-x2) * (y1-y2)

# resolution
[1, 2, 2]
[ z x y ]

In [ ]:
print(